In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from keras import models,layers
from keras import regularizers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.externals import joblib
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from keras.models import load_model
import os
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.calibration import CalibratedClassifierCV
from keras.utils.np_utils import to_categorical

#import preparation,evaluation,logistic,SVC,naive,randomForest,XGboost,ADAboost,NeuralNet,result


iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# 評価関数
# accuracy
score = 'accuracy'


# アルゴリズム
# logisticRegression
# SVC
# naiveBayes
# randomForest
# xgboost
# adaboost
# neuralNet
# KNeighborsClassifier

arg = ['SVC','randomForest','KNeighborsClassifier']


# IDと答えが同じテーブルにないとき
train = list(range(1,X_train[:,0].size+1))
train = pd.concat([pd.DataFrame(train),pd.DataFrame(Y_train[:]),pd.DataFrame(X_train[:])],axis = 1)
train.columns =['ID','answer','gakulen','gakuwide','hanalen','hanawide']

test = list(range(1,X_test[:,0].size+1))
test = pd.concat([pd.DataFrame(test),pd.DataFrame(X_test[:,0:])],axis = 1)
test.columns =['ID','gakulen','gakuwide','hanalen','hanawide']

In [4]:
def flower(num):
    ''' 数字を受け取って、対応する名前を返します。'''
    if num == 0:
        return 'Setosa'
    elif num == 1:
        return 'Veriscolour'
    else:
        return 'Virginica'

train['answer'] = train['answer'].apply(flower)

In [24]:
# データの準備
train_x, train_y, test, test_ID, answer_num = preparation(train, test)

# 正規化
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test = scaler.fit_transform(test)

#　アルゴリズムの結果を格納するリスト
ranking = {}

ranking = evaluation(train_x, train_y, ranking, answer_num, arg)

# アルゴリズムをランキング形式で並べ替え
ranking = sorted(ranking.items(), key=lambda x:-x[1])

# 結果
result(train_x, train_y, test, test_ID, score, answer_num,  ranking)

C:\Users\fujii\Anaconda3\envs\tesorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':
C:\Users\fujii\Anaconda3\envs\tesorflow-gpu\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\fujii\Anaconda3\envs\tesorflow-gpu\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\fujii\Anaconda3\envs\tesorf

評価指標：accuracy

アルゴリズムランキング
1位：ランダムフォレスト（0.9618736383442266）
2位：K近傍法（0.9531590413943355）
3位：サポートベクトルマシン（0.9240273887332711）


In [5]:
def preparation(train, test):
    
    # trainとtestを結合する前に判別フラグを立てる
    train['train_flg'] = 1
    test['train_flg'] = 0
    
    # testに答えの列を追加
    test[train.columns[1]] = 9
    
    # trainとtestを結合
    #train_test_combine = pd.concat([train.drop(train.columns[1], axis=1),test],axis=0)
    train_test_combine = pd.concat([train,test],axis=0)

    # 削除する行名と列名の格納するリストを定義
    delete_row = []
    delete_column = []

    # 欠損値の割合を計算
    percent = 100 * train_test_combine.isnull().sum()/ len(train_test_combine)      

    # 特徴量の数だけループ
    for i in range(percent.size):
        # object型かつカラムに30種類以上のデータが含まれる、または、欠損値が全体の15％以上ある場合列を削除
        if (train_test_combine[percent.index[i]].dtype == ('object') 
        and train_test_combine[percent.index[i]].nunique() >= 30)or percent[i] >= 15:
            delete_column.append(percent.index[i])
        # 欠損値が存在し、全体の15％以下である場合行を削除
        elif percent[i] > 0 and percent[i] < 15:
            delete_row.append(percent.index[i])
    
    # 行の削除
    for i in delete_row:
        train_test_combine = train_test_combine[train_test_combine[i].notnull()]
    # 列の削除
    for j in delete_column:
        train_test_combine = train_test_combine.drop([j], axis=1)
        
    # trainの答えを退避
    train_y = train_test_combine[train_test_combine['train_flg'] == 1][train.columns[1]]
    # testのIDを退避
    test_ID = train_test_combine[train_test_combine['train_flg'] == 0][test.columns[0]]
    # 答えの種類の数を退避
    answer_num = train[train.columns[1]].nunique()
    
    # ID, 答えを削除
    train_test_combine = train_test_combine.drop([train.columns[0],train.columns[1]],axis=1) 
    
    # カテゴリカル変数の特徴量だけを抽出
    categorical_columns = [c for c in train_test_combine.columns if (train_test_combine[c].dtype != ('float64') and train_test_combine[c].dtype != ('int64') and train_test_combine[c].dtype != ('bool'))]

    # カテゴリカル変数の種類と個数を出力し、one-hot-encoding
    for col in categorical_columns:
        ohe = pd.get_dummies(train_test_combine[col],drop_first=True)
        train_test_combine = train_test_combine.drop([col],axis=1)
        train_test_combine = pd.concat([train_test_combine,ohe], axis=1)
        
    train = train_test_combine[train_test_combine['train_flg'] == 1].drop(['train_flg'], axis=1)
    test = train_test_combine[train_test_combine['train_flg'] == 0].drop(['train_flg'], axis=1)
    
    return train, train_y, test, test_ID, answer_num

In [6]:
def evaluation(x, y, ranking, answer_num, arg):

    for i in arg:
        if i == 'logisticRegression':
            ranking = logistic(x, y, ranking)
        elif i == 'SVC':
            ranking = SVCK(x, y, ranking)
        elif i == 'naiveBayes':
            ranking = naive(x, y, ranking)
        elif i == 'randomForest':
            ranking = randomForest(x, y, ranking)
        elif i == 'xgboost':
            ranking = XGboost(x, y, ranking)
        elif i == 'adaboost':
            ranking = ADAboost(x, y, ranking)
        elif i == 'neuralNet':
            ranking = NeuralNet(x, y, answer_num, ranking)
        elif i == 'KNeighborsClassifier':
            ranking = KNeighber(x, y, ranking)
        else:
            print(i + 'は対応していません')
    
    return ranking

In [7]:
def logistic(x, y, ranking):
    #ロジスティック回帰

    logistic_regression = LogisticRegressionCV(random_state=0, penalty='l2', Cs=10, multi_class='ovr',n_jobs=-1)
    answer = cross_val_score(logistic_regression, x, y, scoring = score)
    ranking['ロジスティック回帰'] = np.average(answer)
    return ranking

In [8]:
def SVCK(x, y, ranking):
    #サポートベクトルマシン

    svc = SVC(C=1.0,kernel='linear',probability=True,random_state=0)
    answer = cross_val_score(svc, x, y, scoring = score)
    ranking['サポートベクトルマシン'] = np.average(answer)
    return ranking

In [9]:
def naive(x, y, ranking):
    #ナイーブベイズ

    naive = GaussianNB()
    answer = cross_val_score(naive, x, y, scoring = score)
    ranking['ナイーブベイズ'] = np.average(answer)
    return ranking

In [10]:
def randomForest(x, y, ranking):
    #ランダムフォレスト

    RF = RandomForestClassifier(criterion='gini', n_estimators=25, random_state=0, n_jobs=-1)
    answer = cross_val_score(RF, x, y, scoring = score)
    ranking['ランダムフォレスト'] = np.average(answer)
    return ranking

In [11]:
def XGboost(x, y, ranking):
    #Xgboost

    xgboost = xgb.XGBClassifier()
    answer = cross_val_score(xgboost, x, y, scoring = score)
    ranking['XGブースト'] = np.average(answer)
    return ranking

In [12]:
def ADAboost(x, y, ranking):
    #ADAブースト

    adaboost = AdaBoostClassifier(random_state=0)
    answer = cross_val_score(adaboost, x, y, scoring =score)
    ranking['ADAブースト'] = np.average(answer)
    return ranking

In [13]:
def KNeighber(x, y, ranking):
    #K近傍法

    Knn = KNeighborsClassifier()
    answer = cross_val_score(Knn, x, y, scoring =score)
    ranking['K近傍法'] = np.average(answer)
    return ranking

In [14]:
def NeuralNet(x, y, answer_num, ranking):
    # ニューラルネット

    (X_train, X_test, y_train, y_test) = train_test_split(x, y, test_size=0.3, random_state=0)

    y_train_ohe = pd.get_dummies(y_train,drop_first=False)
    y_test_ohe = pd.get_dummies(y_test,drop_first=False)
    
    model = models.Sequential()
    model.add(layers.Dense(units=32, activation='relu', input_shape=(X_train[0].size,)))
    model.add(layers.Dense(units=64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=answer_num, activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer = "rmsprop", metrics=[score])

    # 早期終了を行うコールバック関数を設定
    callbacks = [EarlyStopping(monitor="val_loss", patience=2)]

    history = model.fit(X_train, y_train_ohe, epochs=20, verbose=0, callbacks = callbacks, batch_size=100) 
    loss, scores = model.evaluate(X_test, y_test_ohe)

    ranking['ニューラルネット'] = scores
    return ranking

In [15]:
def result(x, y, xt, test_ID, score, answer_num, ranking):

    print("評価指標：" + score)
    print()
    print("アルゴリズムランキング")
    j = 1

    for k, i in ranking:
        print(str(j) + "位：" + k,end="")
        print("（" ,end="")
        print(i, end="")
        print("）")
        if j == 1:
            if k == 'ニューラルネット':
                
                y_ohe = pd.get_dummies(y,drop_first=False)

                model = models.Sequential()
                model.add(layers.Dense(units=32, activation='relu', input_shape=(X_train[0].size,)))
                model.add(layers.Dense(units=64, activation='relu'))
                model.add(layers.Dropout(0.2))
                model.add(layers.Dense(units=answer_num, activation='softmax'))
                model.compile(loss="categorical_crossentropy", optimizer = "rmsprop", metrics=[score])

                callbacks = [EarlyStopping(monitor="val_loss", patience=2),
                             ModelCheckpoint(filepath = "best_model.h5",monitor = "val_loss", save_best_only = True)]
                model.fit(x, y_ohe, epochs=20, verbose=0, callbacks = callbacks, batch_size=100, validation_split=0.3) 
                predicted = model.predict(xt)
                submit= pd.concat([test_ID, pd.DataFrame(predicted)], axis = 1)
                if os.path.isfile('best_model.pkl'):
                    os.remove('best_model.pkl')
                
            else:
                if k == 'ロジスティック回帰':
                    model = LogisticRegressionCV(random_state=0, penalty='l2', Cs=10, n_jobs=-1).fit(x,y)
                elif k == 'サポートベクトルマシン':
                    model = SVC(C=1.0,kernel='linear',probability=True,random_state=0).fit(x,y)
                elif k == 'ナイーブベイズ':
                    model = CalibratedClassifierCV(GaussianNB(), cv=2, method='softmax')
                elif k == 'ランダムフォレスト':
                    model = RandomForestClassifier(criterion='gini', n_estimators=25, random_state=0, n_jobs=-1).fit(x,y)
                elif k == 'XGブースト':
                    model = xgb.XGBClassifier().fit(x,y)
                elif k == 'ADAブースト':
                    model = AdaBoostClassifier(random_state=0).fit(x,y)
                elif k == 'K近傍法':
                    model = KNeighborsClassifier().fit(x,y)

                joblib.dump(model, "best_model.pkl")
                predicted = model.predict_proba(xt)
                submit = pd.concat([test_ID, pd.DataFrame(predicted)],axis = 1)
                if os.path.isfile('best_model.h5'):
                    os.remove('best_model.h5')

        j += 1

    submit.to_csv("submit.csv")